# Policies
This notebook shows how to create your own policy. There are two ways to do so:

1. Extend the `FlexPolicy`: this approach is mostly suitable if you just require some minor changes to the default behavior, like creating different order types.
2. Implement the `Policy` from scratch: this approach provides full flexibility on what the policy should do.

In [ ]:
%use roboquant(1.6.0)
Welcome()

## Extending the `FlexPolicy`

We're now ready to develop a "custom policy". For this first example we extend the `FlexPolicy` and create our own Order type by overriding the `createOrder` method. The default behavior is to create a `MarketOrder`, but we change it to a `BracketOrder`.

In [ ]:
class MyPolicy(val trailPercentage: Double = 0.02, val stopPercentage: Double = 0.01) : FlexPolicy() {
    
   
        override fun createOrder(signal: Signal, size: Size, priceAction: PriceAction): Order? {
            
            // In this example we don't short and exit orders are covered by the initial bracket order.
            if (size < 0) return null
            
            val asset = signal.asset
            val price = priceAction.getPrice()
      
            return BracketOrder(
                MarketOrder(asset, size), // Entry Order
                TrailOrder(asset, -size, trailPercentage), // Take profit order
                StopOrder(asset, -size, price * (1.0 - stopPercentage)) // Stop loss order
            )
        }
}

## Create a new roboquant and evaluate it
We are now ready to back-test our strategy using the just created Policy. And suppose we want to find out what is the optimal percentage to use in our policy, we'll iterate over 10 values to see what results in the best solution.

In [ ]:
val feed = AvroFeed.sp500()
val strategy = EMAStrategy()
val policy = MyPolicy()
val roboquant =  Roboquant(strategy, AccountMetric(), policy = policy)
roboquant.run(feed)

In [ ]:
var data = roboquant.logger.getMetric("account.equity")
TimeSeriesChart(data)

# Implementing the `Policy` interface

The following cell shows a very minimalistic implementation of a Policy in which BUY ratings are generating buy market orders and SELL ratings are generating sell market orders if there is an open position.

NOTE: In general it is important that a Policy is very robust, especially if you intent to use it also for live trading. It means that all kind of corner cases need to be catered for. A simple implementation as the one below isn't sufficient. Have a look at the `FlexPolicy` in the roboquant GitHub repo to see a more complete implementation.

In [ ]:
class MyPolicy2 : Policy {
    
    // This is the only method that is required to implement and transforms incoming signals into orders 
    override public fun act(signals: List<Signal>, account: Account, event: Event): List<Order> {
        
        val orders = mutableListOf<Order>()
        
        for (signal in signals) {
            val asset = signal.asset
            if (signal.rating.isPositive) 
                orders.add(MarketOrder(asset, 10))
            else if (signal.rating.isNegative && account.positions.getPosition(asset).long)
                orders.add(MarketOrder(asset, -10))
        }
        
        return orders
    }
    
}

In [ ]:
val strategy = EMAStrategy()
val policy = MyPolicy2()
val roboquant =  Roboquant(strategy, AccountMetric(), policy = policy)
roboquant.run(feed)

In [ ]:
roboquant.broker.account

In [ ]:
var data = roboquant.logger.getMetric("account.equity")
TimeSeriesChart(data)